In [1]:
import pandas as pd 
import numpy as np 
import gc
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
def trim_categories(df, cols_list):
    for col in cols_list:
        df[col].cat.rename_categories(df[col].cat.categories.str.strip(), inplace=True)

### Import data, setting dtype to save memory

In [3]:
levels_keep_cols = ['TankId', 'TankLevelPercent', 'TankLevelGallons', 'ExactFuelEventId']
levels_col_types = {'TankId': 'category',
             'ExactFuelEventId': 'uint64',
             'TankLevelPercent': 'float32',
             'TankLevelGallons': 'float32'}
#Rows with TankId == '2' are dropped on read-in, after being manually identified
levels_fueltank_df = pd.read_csv('data/Extranet2.ExactFuelTankLevels.csv', 
                                 skiprows=[48671, 1824800, 4579415, 5656381],
                                 usecols = levels_keep_cols, 
                                 dtype=levels_col_types ).drop(
    ['TankId'], axis=1)

#levels_fueltank_df = levels_fueltank_df.set_index('ExactFuelEventId').drop(['TankId'], axis=1)

#### Fuel Optimization data; only need tank capacity

In [4]:
optimization_fuel_df = pd.read_csv('data/BGEIDSC.EF2EFFP.csv',
                                   dtype={'TANKCAP': 'uint16', 'TRUCK' : 'uint32'},
                                  usecols = ['TRUCK', 'TANKCAP'])
optimization_fuel_df.columns = [s.lower() for s in optimization_fuel_df.columns]

In [5]:
#trim_categories(optimization_fuel_df, ['truck'])
#optimization_fuel_df.set_index('truck', inplace=True)

In [6]:
units_df = pd.read_csv('data/IBGEFILE.UNITS.csv', 
                       dtype = {'UNMAKE': 'category'})
#Can drop non-int units
units_df.columns = ['unit_number', 'model_year', 'make'] 

# Trim white space from category strings 

# Remove invalid entries
units_df.unit_number = pd.to_numeric(units_df.unit_number, errors = 'coerce')
units_df = units_df[units_df.unit_number != 1].dropna()
#units_df = units_df[units_df.unit_number.isin(['ZEBE', 'CCTRK', 'BGXL', '1']) == False]
#set index

trim_categories(units_df, ['make'])
# Fix typos
units_df.make[units_df.make.isin(['KENOWRTH', 'KEWORTH'])] = 'KENWORTH'


In [7]:
units_df.make.cat.categories

Index(['', 'FREIGHTLINER', 'KENOWRTH', 'KENWORTH', 'KEWORTH', 'MACK',
       'PETERBILT', 'SYSTEM TEST', 'VOLVO'],
      dtype='object')

In [8]:
units_df.make.value_counts()

KENWORTH        526
VOLVO           139
FREIGHTLINER     15
PETERBILT        14
MACK             10
SYSTEM TEST       0
KEWORTH           0
KENOWRTH          0
                  0
Name: make, dtype: int64

In [9]:
transaction_df_cols = ['tr_id', 'tr_date', 'tr_time', 'tr_station_id', 'tr_station_name',
                         'tr_station_cty', 'tr_station_st', 'tr_drvr_code', 'tr_unit_id', 
                         'tr_item_cd1', 'tr_item_qty1', 'tr_item_cd2', 'tr_item_qty2',
                          'tr_item_cd3', 'tr_item_qty3', 'tr_item_cd4', 'tr_item_qty4',
                          'tr_item_cd5', 'tr_item_qty5', 'tr_item_cd6', 'tr_item_qty6']
tr_dtypes= {'tr_id' : 'uint32',  
            'tr_date' : 'uint32', 
            'tr_time' : 'uint32', 
            'tr_station_name' : 'object', 
            'tr_station_cty' : 'object', 
            'tr_station_st' : 'object', 
            'tr_drvr_code' : 'category', 
            'tr_unit_id' : 'category', 
            'tr_item_cd1' : 'category', 
            'tr_item_qty1' : 'float32', 
            'tr_item_cd2' : 'category', 
            'tr_item_qty2' : 'float32', 
            'tr_item_cd3' : 'category', 
            'tr_item_qty3' : 'float32', 
            'tr_item_cd4' : 'category', 
            'tr_item_qty4' : 'float32', 
            'tr_item_cd5' : 'category', 
            'tr_item_qty5' : 'float32', 
            'tr_item_cd6' : 'category', 
            'tr_item_qty6' : 'float32'}
act_fuel_transactions = ['ULSD', 'FUEL', 'CDSL', 'DSL1', 'BDSL']
transaction_df = pd.read_csv('data/BGETCHDATA.PTCHTRANH.csv', names = transaction_df_cols, dtype = tr_dtypes, header=0 )

# Trim white space from category strings 
trim_categories(transaction_df, ['tr_drvr_code','tr_unit_id','tr_item_cd1', 'tr_item_cd2', 'tr_item_cd3', 'tr_item_cd4', 'tr_item_cd5', 'tr_item_cd6' ])

#### Create datetime column in transactions_df

In [10]:
#confirm no junk data in this col (returns false if no NaNs)
None in transaction_df.tr_time

# 20170112 3

#Combine tr_date and tr_time into a single date time column
    #1. zero pad tr_time on left for interpretation as military time
        #cast tr_time as string
transaction_df.tr_time = transaction_df.tr_time.astype(str)

# 20170112 '3' Time column is now a string
#ensure none of my strings are over 4 digits in length
for s in transaction_df.tr_time:
    assert len(s) < 5

#check for blanks
'' in transaction_df.tr_time

#add zeros to the left until each cell is len 4
zero_padded_time = []
for s in transaction_df.tr_time:
    while len(s) < 4:
        s = '0' + s
    zero_padded_time.append(s)
# 20170112 '0003' zero padded minutes
#spot check
for s in zero_padded_time:
    assert len(s) == 4

transaction_df.tr_time = zero_padded_time

#append tr_date (as string) and tr_time together, and then convert them to dateime
#transaction_df.tr_datetime 
tst = [datetime.strptime(str(d) + 
                         t, '%Y%m%d%H%M') 
       for d, t in zip(transaction_df.tr_date,
                       transaction_df.tr_time)]
#love list comprehensions

transaction_df['tr_datetime'] = tst#assign

#drop superfluous columns
transaction_df.drop(['tr_date','tr_time'], axis = 'columns', inplace=True)
transaction_df.head()

,tr_id,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,tr_item_qty1,tr_item_cd2,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6,tr_datetime
0,42574765,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,1.000000,,0.00,,0.0,,0.0,,0.0,,0.0,2017-01-01 00:20:00
1,42575331,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,137.399994,DEFD,7.71,CADV,1.0,,0.0,,0.0,,0.0,2017-01-01 00:48:00
2,42578678,516277,PILOT 421,DALTON,GA,RUSR,1713,ULSD,137.929993,DEFD,4.12,,0.0,,0.0,,0.0,,0.0,2017-01-01 01:34:00
3,42578722,231100,FJ-BRUNSWICK 627,BRUNSWICK,GA,FOSJA,1711,ULSD,45.000000,,0.00,,0.0,,0.0,,0.0,,0.0,2017-01-01 01:37:00
4,42579695,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,HICWIL,1837,ULSD,107.360001,,0.00,,0.0,,0.0,,0.0,,0.0,2017-01-01 03:02:00


In [11]:
transaction_df.tr_item_cd2.value_counts()

        96333
DEFD    42468
CADV     8196
ADD       348
OIL       264
SCLE      236
UREA       15
WWFL        1
Name: tr_item_cd2, dtype: int64

In [12]:
# These rows are dropped
transaction_df[transaction_df.tr_unit_id.isin(['OFFICE', '']) == True]

,tr_id,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,tr_item_qty1,tr_item_cd2,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6,tr_datetime
1174,42977478,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,SPED,OFFICE,ULSD,50.000000,,0.0,,0.0,,0.0,,0.0,,0.0,2017-01-05 07:40:00
1383,43037746,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,SPED,OFFICE,ULSD,130.570007,,0.0,,0.0,,0.0,,0.0,,0.0,2017-01-05 14:57:00
6812,44866479,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,HILB,,ULSD,60.790001,,0.0,,0.0,,0.0,,0.0,,0.0,2017-01-21 06:08:00
6955,44908231,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,HILB,,ULSD,30.440001,,0.0,,0.0,,0.0,,0.0,,0.0,2017-01-21 14:33:00
13493,47176337,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,SMIFR,,ULSD,96.269997,,0.0,,0.0,,0.0,,0.0,,0.0,2017-02-09 09:11:00
15203,47747773,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,SMIFR,,ULSD,62.840000,,0.0,,0.0,,0.0,,0.0,,0.0,2017-02-14 10:46:00
16503,48174487,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,SMIFR,,ULSD,82.480003,,0.0,,0.0,,0.0,,0.0,,0.0,2017-02-17 11:53:00
16583,48198811,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,SPED,OFFICE,ULSD,50.000000,,0.0,,0.0,,0.0,,0.0,,0.0,2017-02-17 15:01:00
17310,48478526,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,1974,,ULSD,10.060000,,0.0,,0.0,,0.0,,0.0,,0.0,2017-02-20 12:12:00
18077,48727766,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,SMIFR,,ULSD,88.010002,,0.0,,0.0,,0.0,,0.0,,0.0,2017-02-22 09:42:00


In [13]:
# Drops Office and blank Unit IDs
#transaction_df = transaction_df[transaction_df.tr_unit_id.isin(['OFFICE', '      ']) == False]

In [14]:
events_keep_cols = ['Id', 'EventTimeStamp', 'EquipmentID', 
                    'DriverID', 'Latitude', 'Longitude', 'LocationTimeStamp', 
                    'Speed', 'Odometer', 'IgnitionStatus', 'EFReportReason']
events_col_types = {'DriverID': 'category',
             'EFReportReason': 'category',
             'EquipmentID': 'uint32',
             'Id': 'uint64',
             'IgnitionStatus': 'category',
             'Latitude': 'float64',
             'Longitude': 'float64',
             'Odometer': 'float64',
             'Speed': 'uint16'}

events_levels_df = pd.read_csv('data/Extranet2.ExactFuelEvents.csv', 
                             usecols=events_keep_cols, 
                            dtype=events_col_types, 
                             parse_dates=['EventTimeStamp', 'LocationTimeStamp'], 
                            infer_datetime_format=True).merge(
    levels_fueltank_df, copy=False, how='inner', left_on='Id', right_on='ExactFuelEventId').merge(
    optimization_fuel_df, copy=False, how='left', left_on='EquipmentID', right_on = 'truck')
trim_categories(events_levels_df, ['DriverID','EFReportReason', 'IgnitionStatus' ])

In [15]:
'''events_df = pd.read_csv('data/Extranet2.ExactFuelEvents.csv', 
                             usecols=events_keep_cols, 
                            dtype=events_col_types, 
                             index_col = 'EventTimeStamp',
                             parse_dates=['EventTimeStamp', 'LocationTimeStamp'], 
                            infer_datetime_format=True)
trim_categories(events_df, ['DriverID','EFReportReason', 'IgnitionStatus' ])
'''

"events_df = pd.read_csv('data/Extranet2.ExactFuelEvents.csv', \n                             usecols=events_keep_cols, \n                            dtype=events_col_types, \n                             index_col = 'EventTimeStamp',\n                             parse_dates=['EventTimeStamp', 'LocationTimeStamp'], \n                            infer_datetime_format=True)\ntrim_categories(events_df, ['DriverID','EFReportReason', 'IgnitionStatus' ])\n"

In [16]:
# Save 220 MB memory and 
levels_fueltank_df.index = levels_fueltank_df.index.astype('uint64')
gc.collect()

2828

In [17]:
ls data/

BGEIDSC.EF2EFFP.csv                  Extranet2.QCPerformanceExtracts.csv*
BGETCHDATA.PTCHTRANH.csv*            IBGEFILE.UNITS.csv*
Extranet2.ExactFuelEvents.csv*       t.4.01.0.jpg.png
Extranet2.ExactFuelTankLevels.csv*


In [18]:
performance_df = pd.read_csv('data/Extranet2.QCPerformanceExtracts.csv',
                            parse_dates=['EventTimeStamp','LocationTimeStamp', 'DataStartTimeStamp','DataEndTimeStamp' ], 
                            infer_datetime_format=True)
# Remove invalid Truck IDs
performance_df = performance_df[performance_df.EquipmentID < 9999]

ValueError: 'EventTimeStamp' is not in list

In [ ]:
performance_df[performance_df.RegisteredDriver == 0].DriverID.value_counts()

In [ ]:
perf_cols = ['Id', 'EventTimeStamp', 'EquipmentID', 'DriverID', 'IgnitionStatus', 'TripStatus',
       'DataStartTimeStamp', 'DataEndTimeStamp', 'Distance', 'DriveTime',
       'EngineTime', 'MoveTime', 'IntertripIdleTime', 'ShortIdleTime',
       'OverSpeedTime', 'TotalFuelUsed', 'IdleFuelUsed',
        'CruiseControlTime', 'TopGearTime',
       'GearDataSource', 'SpeedUnits', 'LocationTimeStamp', 'Latitude', 'Longitude', 
            'FaultFlag', 'RegisteredDriver',]
performance_df.columns

In [ ]:
df_names_list = ['events_levels_df', 'levels_fueltank_df', 'optimization_fuel_df', 'units_df', 'transaction_df', 'performance_df']
df_list = [events_levels_df, levels_fueltank_df, optimization_fuel_df, units_df, transaction_df, performance_df]
df_dict = dict(zip(df_names_list, df_list))

for k, v in df_dict.items():
    print(k + ' shape: ' + str(v.shape))

In [ ]:
events_levels_df.head()

#### Events grouped by Driver and Truck

In [ ]:
dt_grouped = events_levels_df.groupby(['DriverID', 'EquipmentID'])

#### Sizes of groups as a dataframe

In [ ]:
dt_sizes = dt_grouped.size().to_frame('events').sort_values('events', ascending=False)

In [ ]:
dt_sizes.shape

In [ ]:
dt_sizes[dt_sizes.events < 20].shape[0]

In [ ]:
sns.set_style('whitegrid')
plt.rcParams["patch.force_edgecolor"] = True
dt_sizes[dt_sizes.events > 20].plot(kind='hist', bins=30)
plt.xlabel('Events grouped by Driver-Truck pairs')
plt.show()

In [ ]:
dt_sizes.head(50).sort_index()

### Sample data for specific drivers

In [ ]:
sample_drivers = ['EDWTI']
sample_trucks = ['2014']
events_col_list = ['EventTimeStamp', 'Odometer', 'TankLevelPercent', 'IgnitionStatus', 'EFReportReason',
                   'Speed',  'TankLevelGallons', 'Latitude', 'Longitude', 'LocationTimeStamp',
                  'tankcap', 'EquipmentID', 'DriverID']
sample_bool_series = np.logical_and(events_levels_df.DriverID.isin(sample_drivers), 
                                   events_levels_df.EquipmentID.isin(sample_trucks))
drivers_events = events_levels_df[sample_bool_series].loc[:, events_col_list].set_index('EventTimeStamp')

In [ ]:
drivers_events.info()

In [ ]:
'''event_col_list = ['Odometer',  'IgnitionStatus', 'EFReportReason',
                   'Speed',  'Latitude', 'Longitude', 'LocationTimeStamp',
                  'EquipmentID', 'DriverID']
driver_event_filter = np.logical_and(events_df.DriverID.isin(sample_drivers), 
                                   events_df.EquipmentID.isin(sample_trucks))
drivers_events_full = events_df[driver_event_filter].loc[:, event_col_list]'''

In [ ]:
drivers_events.loc['2017-01-12 05:56': '2017-01-12 12:15']

In [ ]:
'''events_df.info(memory_usage='deep')'''

In [ ]:
'''#drivers_events_full.info()
len(drivers_events_full.loc['2017-01-12'])
#drivers_events_full.head()'''

In [ ]:
sample_drivers = ['EDWTI']
sample_trucks = ['2014']
trans_col_list = ['tr_datetime','tr_item_cd1', 'tr_item_qty1', 'tr_station_id', 
                  'tr_station_name','tr_station_cty', 'tr_station_st', 'tr_drvr_code', 'tr_unit_id']
    

sample_tran_bool_series = np.logical_and(transaction_df.tr_drvr_code.isin(sample_drivers), 
                                   transaction_df.tr_unit_id.isin(sample_trucks))
drivers_trans = transaction_df[sample_tran_bool_series].loc[:, trans_col_list].sort_values(['tr_datetime'])

In [ ]:
transaction_df.columns

In [ ]:
drivers_trans[:4]

## Cleaning up odometer data in evets_levels_df

In [ ]:
events_levels_df.head()

In [ ]:
#Are there bad odometer readings?
events_levels_df.info()

In [ ]:
#well at least they're all floats. I guess I could try a histogram
_ = plt.hist(events_levels_df.Odometer)
plt.show()

### Approach 1: Intervals between transactions 


### Approach 2: Max Odometer between transaction timestamps
#### Outlier odometer readings: Use diff method for decreases in odometer 